In [42]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft  import EdgeChromiumDriverManager
from selenium.webdriver.support.wait import WebDriverWait
from datetime import datetime
import time
import csv

In [43]:
# get browser, maximise windows, open web page
browser = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()))
browser.maximize_window()
browser.get('https://www.tripadvisor.com/Hotel_Review-g187849-d2340336-Reviews-Armani_Hotel-Milan_Lombardy.html')
# wait for cookies button, click accept
cookies_wait = WebDriverWait(browser, timeout=3)
cookies_accept = cookies_wait.until(lambda b: b.find_element('xpath','//*[@id="onetrust-accept-btn-handler"]'))
cookies_accept.click()
expand = browser.find_element('xpath', ".//div[contains(@data-test-target,'expand-review')]")
if expand:
    expand.click()
reviews = browser.find_elements('xpath', ".//div[@data-reviewid]")
reviews


[<selenium.webdriver.remote.webelement.WebElement (session="7f6bd9672a08b4fbdda4d868bed69365", element="D92AA030C7443450A78A3A57E43D9D2D_element_59")>,
 <selenium.webdriver.remote.webelement.WebElement (session="7f6bd9672a08b4fbdda4d868bed69365", element="D92AA030C7443450A78A3A57E43D9D2D_element_60")>,
 <selenium.webdriver.remote.webelement.WebElement (session="7f6bd9672a08b4fbdda4d868bed69365", element="D92AA030C7443450A78A3A57E43D9D2D_element_61")>,
 <selenium.webdriver.remote.webelement.WebElement (session="7f6bd9672a08b4fbdda4d868bed69365", element="D92AA030C7443450A78A3A57E43D9D2D_element_62")>,
 <selenium.webdriver.remote.webelement.WebElement (session="7f6bd9672a08b4fbdda4d868bed69365", element="D92AA030C7443450A78A3A57E43D9D2D_element_63")>,
 <selenium.webdriver.remote.webelement.WebElement (session="7f6bd9672a08b4fbdda4d868bed69365", element="D92AA030C7443450A78A3A57E43D9D2D_element_64")>,
 <selenium.webdriver.remote.webelement.WebElement (session="7f6bd9672a08b4fbdda4d868bed6

In [44]:
with open('ArmaniHotelReviews.csv','w',encoding='utf-8') as csvFile:
    csvWriter = csv.writer(csvFile)
    for i in range(10):
        time.sleep(6)
        
        for review in reviews:
            rating_rubble = review.find_element('xpath',".//span[contains(@class, 'ui_bubble_rating bubble_')]")
            score = rating_rubble.get_attribute("class").split("_")[3]

            date_ob = review.find_element('xpath',".//span[@class=teHYY _R Me S4 H3']")
            date_rec = date_ob.text.replace("Date of stay:",'')
            date = datetime.strptime(date_rec,'%B %Y')
            title = review.find_element('xpath', ".//div[contains(@data-test-target,'review-title')]").text
            comment = review.find_element('xpath',".//span[@class = 'QewHA H4 _a']").text.replace("\n","  ")
            csvWriter.writerow([score,date.year,date.month,title,comment])
        browser.find_element('xpath', './/a[@class="ui_button nav next primary "]').click()

InvalidSelectorException: Message: invalid selector: Unable to locate an element with the xpath expression .//span[@class=teHYY _R Me S4 H3'] because of the following error:
SyntaxError: Failed to execute 'evaluate' on 'Document': The string './/span[@class=teHYY _R Me S4 H3']' is not a valid XPath expression.
  (Session info: MicrosoftEdge=116.0.1938.81); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalid-selector-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7B8D73D92+61490]
	Microsoft::Applications::Events::EventProperty::~EventProperty [0x00007FF7B8D05FE2+740626]
	(No symbol) [0x00007FF7B8ADB8AE]
	(No symbol) [0x00007FF7B8AE0340]
	(No symbol) [0x00007FF7B8AE2829]
	(No symbol) [0x00007FF7B8AE2910]
	(No symbol) [0x00007FF7B8B22FDB]
	(No symbol) [0x00007FF7B8B23405]
	(No symbol) [0x00007FF7B8B1709C]
	(No symbol) [0x00007FF7B8B4394F]
	(No symbol) [0x00007FF7B8B17021]
	(No symbol) [0x00007FF7B8B16ECD]
	(No 